#Tutorial Learning-to-Rank dengan LambdaMART
Alfan F. Wicaksono (Fasilkom UI)

# Training Dataset
Kita gunakan dataset untuk learning-to-rank pada domain Medical Information Retrieval yang dapat diakses di: 

https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/

Data dapat diunduh di:

https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz

Setelah diunduh, data kemudian diekstrak, dan pada tutorial kali ini, kita hanya tertarik pada tiga file (tab-separated format) untuk melakukan training sebuah model LambdaMART:


1.   train.docs --> koleksi 3612 dokumen medis (FORMAT: DOC_ID <tab> CONTENT)
2.   train.3-2-1.qrel --> relevant judgments (FORMAT: QUERY_ID <tab> 0 <tab> DOC_ID <tab> RELEVANCE_LEVEL)
3.   train.vid-desc.queries --> 812 query (FORMAT: QUERY_ID <tab> QUERY_TEXT)




In [ ]:
!wget -c https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz -P data
!tar -xvf data/nfcorpus.tar.gz

--2022-11-06 14:17:43--  https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz
Resolving www.cl.uni-heidelberg.de (www.cl.uni-heidelberg.de)... 147.142.207.78
Connecting to www.cl.uni-heidelberg.de (www.cl.uni-heidelberg.de)|147.142.207.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31039523 (30M) [application/x-gzip]
Saving to: ‘data/nfcorpus.tar.gz’

nfcorpus.tar.gz     100%[===================>]  29.60M  12.1MB/s    in 2.4s    

2022-11-06 14:17:48 (12.1 MB/s) - ‘data/nfcorpus.tar.gz’ saved [31039523/31039523]

nfcorpus/
nfcorpus/train.docs
nfcorpus/test.docs
nfcorpus/dev.docs
nfcorpus/dev.3-2-1.qrel
nfcorpus/test.3-2-1.qrel
nfcorpus/train.3-2-1.qrel
nfcorpus/raw/
nfcorpus/raw/doc_dump.txt
nfcorpus/raw/dev.docs.ids
nfcorpus/raw/dev.queries.ids
nfcorpus/raw/test.docs.ids
nfcorpus/raw/test.queries.ids
nfcorpus/raw/train.docs.ids
nfcorpus/raw/train.queries.ids
nfcorpus/raw/stopwords.large
nfcorpus/raw/nfdump.txt
nfcorpus/raw/all_videos.ids

In [ ]:
# training data yang sudah terekstraksi ada di direktori nfcorpus

# intip 10 baris pertama di train.docs
!head -10 nfcorpus/train.docs

MED-10	statin breast cancer survival nationwide cohort study finland abstract recent studies suggested statins established drug group prevention cardiovascular mortality delay prevent breast cancer recurrence effect disease-specific mortality remains unclear evaluated risk breast cancer death statin users population-based cohort breast cancer patients study cohort included newly diagnosed breast cancer patients finland num num num cases identified finnish cancer registry information statin diagnosis obtained national prescription database cox proportional hazards regression method estimate mortality statin users statin time-dependent variable total num participants statins median follow-up num years diagnosis range num num years num participants died num num due breast cancer adjustment age tumor characteristics treatment selection post-diagnostic pre-diagnostic statin lowered risk breast cancer death hr num num ci num num hr num num ci num num risk decrease post-diagnostic statin affe

In [ ]:
!head -10 nfcorpus/train.3-2-1.qrel

PLAIN-3	0	MED-2436	3
PLAIN-3	0	MED-2437	3
PLAIN-3	0	MED-2438	3
PLAIN-3	0	MED-2439	3
PLAIN-3	0	MED-2440	3
PLAIN-3	0	MED-2427	2
PLAIN-3	0	MED-2428	2
PLAIN-3	0	MED-2429	2
PLAIN-3	0	MED-2430	2
PLAIN-3	0	MED-2431	2


In [ ]:
!head -10 nfcorpus/train.vid-desc.queries 

PLAIN-2427	diet and exercise synergize to improve endothelial function , the ability of our arteries to relax normally .
PLAIN-2428	the parable of the tiny parachute explains the study that found no relationship between dietary fiber intake and diverticulosis .
PLAIN-2431	pbde fire retardant chemicals in the food supply may contribute to attention and cognitive deficits in children .
PLAIN-2432	peppermint essential oil should be considered the first-line treatment for ibs .
PLAIN-2433	the reversal of blindness due to hypertension and diabetes with dr. kempner ’ s rice and fruit diet demonstrates the power of diet to exceed the benefits of the best modern medicine and surgery has to offer .
PLAIN-2434	squatting and leaning can help straighten the anorectal angle , but a healthy enough diet should make bowel movements effortless regardless of positioning .
PLAIN-2435	most people have between 3 bowel movements a day and 3 a week , but normal doesn ’ t necessarily mean optimal .
PLAIN-2436

In [ ]:
documents = {}
with open("nfcorpus/train.docs") as file:
  for line in file:
    doc_id, content = line.split("\t")
    documents[doc_id] = content.split()

# test untuk melihat isi dari 2 dokumen
print(documents["MED-329"])
print(documents["MED-330"])

['phosphate', 'vascular', 'toxin', 'pubmed', 'ncbi', 'abstract', 'elevated', 'phosphate', 'levels', 'advanced', 'renal', 'failure', 'dysregulated', 'calcium', 'parathyroid', 'hormone', 'vitamin', 'levels', 'contribute', 'complex', 'chronic', 'kidney', 'disease-mineral', 'bone', 'disease', 'ckd-mbd', 'converging', 'evidence', 'vitro', 'clinical', 'epidemiological', 'studies', 'suggest', 'increased', 'vascular', 'calcification', 'mortality', 'vessels', 'exposed', 'high', 'conditions', 'vitro', 'develop', 'apoptosis', 'convert', 'bone-like', 'cells', 'develop', 'extensive', 'calcification', 'clinical', 'studies', 'children', 'dialysis', 'show', 'high', 'increased', 'vessel', 'wall', 'thickness', 'arterial', 'stiffness', 'coronary', 'calcification', 'epidemiological', 'studies', 'adult', 'dialysis', 'patients', 'demonstrate', 'significant', 'independent', 'association', 'raised', 'mortality', 'importantly', 'raised', 'cardiovascular', 'pre-dialysis', 'ckd', 'subjects', 'normal', 'renal', '

In [ ]:
queries = {}
with open("nfcorpus/train.vid-desc.queries") as file:
  for line in file:
    q_id, content = line.split("\t")
    queries[q_id] = content.split()

# test untuk melihat isi dari 2 query
print(queries["PLAIN-2428"])
print(queries["PLAIN-2435"])

['the', 'parable', 'of', 'the', 'tiny', 'parachute', 'explains', 'the', 'study', 'that', 'found', 'no', 'relationship', 'between', 'dietary', 'fiber', 'intake', 'and', 'diverticulosis', '.']
['most', 'people', 'have', 'between', '3', 'bowel', 'movements', 'a', 'day', 'and', '3', 'a', 'week', ',', 'but', 'normal', 'doesn', '’', 't', 'necessarily', 'mean', 'optimal', '.']


In [ ]:
# melalui qrels, kita akan buat sebuah dataset untuk training
# LambdaMART model dengan format
#
# [(query_text, document_text, relevance), ...]
#
# relevance awalnya bernilai 1, 2, 3 --> tidak perlu dinormalisasi
# biarkan saja integer (syarat dari library LightGBM untuk
# LambdaRank)
#
# relevance level: 3 (fully relevant), 2 (partially relevant), 1 (marginally relevant)
import random

NUM_NEGATIVES = 1

q_docs_rel = {} # grouping by q_id terlebih dahulu
with open("nfcorpus/train.3-2-1.qrel") as file:
  for line in file:
    q_id, _, doc_id, rel = line.split("\t")
    if (q_id in queries) and (doc_id in documents):
      if q_id not in q_docs_rel:
        q_docs_rel[q_id] = []
      q_docs_rel[q_id].append((doc_id, int(rel)))

# group_qid_count untuk model LGBMRanker
group_qid_count = []
dataset = []
for q_id in q_docs_rel:
  docs_rels = q_docs_rel[q_id]
  group_qid_count.append(len(docs_rels) + NUM_NEGATIVES)
  for doc_id, rel in docs_rels:
    dataset.append((queries[q_id], documents[doc_id], rel))
  # tambahkan satu negative (random sampling saja dari documents)
  dataset.append((queries[q_id], random.choice(list(documents.values())), 0))

# test
print("number of Q-D pairs:", len(dataset))
print("group_qid_count:", group_qid_count)
assert sum(group_qid_count) == len(dataset), "ada yang salah"
print(dataset[:2])

number of Q-D pairs: 28277
group_qid_count: [102, 16, 48, 68, 37, 17, 32, 37, 28, 64, 22, 35, 74, 73, 66, 30, 46, 48, 62, 71, 58, 90, 63, 43, 60, 51, 71, 54, 32, 40, 36, 44, 85, 38, 35, 83, 62, 56, 37, 42, 49, 96, 41, 32, 14, 50, 22, 14, 18, 26, 65, 25, 41, 81, 42, 39, 58, 23, 110, 25, 37, 67, 62, 44, 53, 51, 15, 42, 52, 76, 20, 52, 103, 53, 58, 25, 34, 19, 33, 25, 36, 69, 106, 58, 82, 17, 48, 92, 72, 18, 65, 94, 82, 36, 42, 52, 39, 24, 42, 45, 50, 50, 59, 8, 36, 66, 28, 50, 48, 44, 22, 90, 25, 56, 59, 39, 34, 39, 55, 34, 20, 21, 44, 29, 31, 51, 39, 105, 52, 57, 107, 75, 45, 40, 72, 45, 53, 53, 50, 120, 38, 65, 84, 48, 50, 66, 21, 61, 27, 38, 84, 66, 42, 11, 46, 68, 16, 33, 85, 50, 65, 50, 60, 21, 41, 29, 53, 92, 12, 16, 63, 42, 48, 43, 5, 45, 21, 47, 35, 31, 40, 26, 43, 18, 35, 21, 77, 10, 79, 45, 25, 47, 24, 38, 11, 34, 26, 32, 25, 12, 13, 44, 22, 29, 26, 45, 23, 22, 24, 38, 49, 18, 42, 64, 26, 49, 80, 84, 56, 50, 83, 44, 65, 13, 43, 29, 20, 28, 71, 82, 43, 84, 42, 50, 24, 56, 41, 24

# Installing Library

In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import lightgbm as lgb
import numpy as np

from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.corpora import Dictionary

# Building LSI/LSA Model
Kita akan buat Latent Semantic Indexing/Analysis (LSI/A) model untuk representasi dokumen menggunakan semua 3612 dokumen yang ada di train.docs. 812 queries yang ada di train.vid-desc.queries nantinya akan dipetakan ke dimensi space TF-IDF yang dibangun dari koleksi 3612 dokumen sebelumnya.

In [ ]:
# bentuk dictionary, bag-of-words corpus, dan kemudian Latent Semantic Indexing
# dari kumpulan 3612 dokumen.
NUM_LATENT_TOPICS = 200

dictionary = Dictionary()
bow_corpus = [dictionary.doc2bow(doc, allow_update = True) for doc in documents.values()]
model = LsiModel(bow_corpus, num_topics = NUM_LATENT_TOPICS) # 200 latent topics

# test melihat representasi vector dari sebuah dokumen & query
def vector_rep(text):
  rep = [topic_value for (_, topic_value) in model[dictionary.doc2bow(text)]]
  return rep if len(rep) == NUM_LATENT_TOPICS else [0.] * NUM_LATENT_TOPICS

print(vector_rep(documents["MED-329"]))
print(vector_rep(queries["PLAIN-2435"]))

[5.9207874649414505, -3.1358543082589088, -2.6077567003839732, 0.6843319142545866, -1.8058665544513566, -2.381580532138159, -1.4732444657643051, -0.27873127872227144, 0.5551310779832449, -1.3395383197884914, -0.7153786683992718, 0.4581364420247896, -1.6566098843650232, -0.2913277536833165, -1.7412154350070537, 0.7543792588118651, 0.7469511741255683, 1.0829986220643602, -1.063502185815146, -1.0675383991453011, -0.1631987326710983, 1.7016046961678888, -1.4945021328851116, -1.514737664397233, -0.5818601019597628, 1.562891894908965, 0.9544145993805866, 0.027280683764285833, 0.12542366495340607, 0.4233737586937925, -0.10745374857244488, -1.5077519876868772, -0.4904822510102606, -1.4728540784057986, 1.2156064339537072, -0.1407798072981922, 0.9517791544423931, -1.9355748215425737, -1.2736212604711388, 0.3394980986622505, -1.533577326385445, -0.03246346472893277, 0.08378221760756216, -0.4696648243618851, 0.3525599514475394, -1.2609080061958933, 1.0128613610507977, 0.759220720711047, 0.89125368

In [ ]:
# kita ubah dataset menjadi terpisah X dan Y
# dimana X adalah representasi gabungan query+document,
# dan Y adalah label relevance untuk query dan document tersebut.
# 
# Bagaimana cara membuat representasi vector dari gabungan query+document?
# cara simple = concat(vector(query), vector(document)) + informasi lain
# informasi lain -> cosine distance & jaccard similarity antara query & doc

from scipy.spatial.distance import cosine

def features(query, doc):
  v_q = vector_rep(query)
  v_d = vector_rep(doc)
  q = set(query)
  d = set(doc)
  cosine_dist = cosine(v_q, v_d)
  jaccard = len(q & d) / len(q | d)
  return v_q + v_d + [jaccard] + [cosine_dist]

X = []
Y = []
for (query, doc, rel) in dataset:
  X.append(features(query, doc))
  Y.append(rel)

# ubah X dan Y ke format numpy array
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


(28277, 402)
(28277,)


# Training the Ranker

In [ ]:
import lightgbm

ranker = lightgbm.LGBMRanker(
                    objective="lambdarank",
                    boosting_type = "gbdt",
                    n_estimators = 100,
                    importance_type = "gain",
                    metric = "ndcg",
                    num_leaves = 40,
                    learning_rate = 0.02,
                    max_depth = -1)

# di contoh kali ini, kita tidak menggunakan validation set
# jika ada yang ingin menggunakan validation set, silakan saja
ranker.fit(X, Y,
           group = group_qid_count,
           verbose = 10)

LGBMRanker(importance_type='gain', learning_rate=0.02, metric='ndcg',
           num_leaves=40, objective='lambdarank')

In [ ]:
# test, prediksi terhadap training data itu sendiri
ranker.predict(X)

array([ 0.44023975, -0.04713903,  0.34965638, ..., -0.1196364 ,
       -0.08329867, -0.26815327])

# Predict Ranking for Unseen Q-D Paris

In [ ]:
query = "how much cancer risk can be avoided through lifestyle change ?"

docs =[("D1", "dietary restriction reduces insulin-like growth factor levels modulates apoptosis cell proliferation tumor progression num defici pubmed ncbi abstract diet contributes one-third cancer deaths western world factors diet influence cancer elucidated reduction caloric intake dramatically slows cancer progression rodents major contribution dietary effects cancer insulin-like growth factor igf-i lowered dietary restriction dr humans rats igf-i modulates cell proliferation apoptosis tumorigenesis mechanisms protective effects dr depend reduction multifaceted growth factor test hypothesis igf-i restored dr ascertain lowering igf-i central slowing bladder cancer progression dr heterozygous num deficient mice received bladder carcinogen p-cresidine induce preneoplasia confirmation bladder urothelial preneoplasia mice divided groups ad libitum num dr num dr igf-i igf-i/dr serum igf-i lowered num dr completely restored igf-i/dr-treated mice recombinant igf-i administered osmotic minipumps tumor progression decreased dr restoration igf-i serum levels dr-treated mice increased stage cancers igf-i modulated tumor progression independent body weight rates apoptosis preneoplastic lesions num times higher dr-treated mice compared igf/dr ad libitum-treated mice administration igf-i dr-treated mice stimulated cell proliferation num fold hyperplastic foci conclusion dr lowered igf-i levels favoring apoptosis cell proliferation ultimately slowing tumor progression mechanistic study demonstrating igf-i supplementation abrogates protective effect dr neoplastic progression"), 
       ("D2", "study hard as your blood boils"), 
       ("D3", "processed meats risk childhood leukemia california usa pubmed ncbi abstract relation intake food items thought precursors inhibitors n-nitroso compounds noc risk leukemia investigated case-control study children birth age num years los angeles county california united states cases ascertained population-based tumor registry num num controls drawn friends random-digit dialing interviews obtained num cases num controls food items principal interest breakfast meats bacon sausage ham luncheon meats salami pastrami lunch meat corned beef bologna hot dogs oranges orange juice grapefruit grapefruit juice asked intake apples apple juice regular charcoal broiled meats milk coffee coke cola drinks usual consumption frequencies determined parents child risks adjusted risk factors persistent significant associations children's intake hot dogs odds ratio num num percent confidence interval ci num num num hot dogs month trend num fathers intake hot dogs num ci num num highest intake category trend num evidence fruit intake provided protection results compatible experimental animal literature hypothesis human noc intake leukemia risk potential biases data study hypothesis focused comprehensive epidemiologic studies warranted"), 
       ("D4", "long-term effects calorie protein restriction serum igf num igfbp num concentration humans summary reduced function mutations insulin/igf-i signaling pathway increase maximal lifespan health span species calorie restriction cr decreases serum igf num concentration num protects cancer slows aging rodents long-term effects cr adequate nutrition circulating igf num levels humans unknown report data long-term cr studies num num years showing severe cr malnutrition change igf num igf num igfbp num ratio levels humans contrast total free igf num concentrations significantly lower moderately protein-restricted individuals reducing protein intake average num kg num body weight day num kg num body weight day num weeks volunteers practicing cr resulted reduction serum igf num num ng ml num num ng ml num findings demonstrate unlike rodents long-term severe cr reduce serum igf num concentration igf num igfbp num ratio humans addition data provide evidence protein intake key determinant circulating igf num levels humans suggest reduced protein intake important component anticancer anti-aging dietary interventions"), 
       ("D5", "cancer preventable disease requires major lifestyle abstract year num million americans num million people worldwide expected diagnosed cancer disease commonly believed preventable num num cancer cases attributed genetic defects remaining num num roots environment lifestyle lifestyle factors include cigarette smoking diet fried foods red meat alcohol sun exposure environmental pollutants infections stress obesity physical inactivity evidence cancer-related deaths num num due tobacco num num linked diet num num due infections remaining percentage due factors radiation stress physical activity environmental pollutants cancer prevention requires smoking cessation increased ingestion fruits vegetables moderate alcohol caloric restriction exercise avoidance direct exposure sunlight minimal meat consumption grains vaccinations regular check-ups review present evidence inflammation link agents/factors cancer agents prevent addition provide evidence cancer preventable disease requires major lifestyle")]

# sekedar pembanding, ada bocoran: D3 & D5 relevant, D1 & D4 partially relevant, D2 tidak relevan

# bentuk ke format numpy array
X_unseen = []
for doc_id, doc in docs:
  X_unseen.append(features(query.split(), doc.split()))

X_unseen = np.array(X_unseen)

In [ ]:
# hitung scores
scores = ranker.predict(X_unseen)
print(scores)

[-0.09123152 -0.37380743 -0.20962416 -0.1076393   0.00553489]


In [ ]:
# Ranking pada SERP

# sekedar pembanding, ada bocoran: D3 & D5 relevant, D1 & D4 partially relevant, D2 tidak relevan
# apakah LambdaMART berhasil merefleksikan hal ini?

did_scores = [x for x in zip([did for (did, _) in docs], scores)]
sorted_did_scores = sorted(did_scores, key = lambda tup: tup[1], reverse = True)

print("query        :", query)
print("SERP/Ranking :")
for (did, score) in sorted_did_scores:
  print(did, score)

query        : how much cancer risk can be avoided through lifestyle change ?
SERP/Ranking :
D5 0.005534888821929053
D1 -0.09123151703233041
D4 -0.10763929796793502
D3 -0.2096241560078664
D2 -0.3738074251668168
